In [1]:
from langchain.llms import OpenAI

In [8]:
import os
os.environ["OPENAI_API_KEY"] = "Replace your key"

llm = OpenAI(temperature=0.6)


In [9]:
text = "What is the capital of India"
print(llm.predict(text))



The capital of India is New Delhi.


## Using opensource model from huggingface 

In [15]:
os.environ["HUGGINGFACEHUB_API_TOKEN"]="Your key"
from langchain import HuggingFaceHub
llm_huggingface=HuggingFaceHub(repo_id="google/flan-t5-large", model_kwargs={"temperature": 0, "max_length": 64})


In [16]:
print(llm_huggingface.predict("Who is the prime minister of india"))

Narendra Modi


### Prompt Templates

In [21]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate(input_variables=['country'],
    template = "Tell me the capital of this {country}")

prompt_template.format(country="India")

'Tell me the capital of this India'

In [25]:
# llm.predict(prompt_template = prompt_template, text = ["India"]) # this line will give us error

from langchain.chains import LLMChain


In [27]:
chain = LLMChain(llm=llm,prompt=prompt_template)
print(chain.run("India"))



The capital of India is New Delhi.


## Combining Multiple Chains Using Simple Sequential Chain

In [29]:
capital_template = PromptTemplate(input_variables=['country'],
    template = "Tell me the capital of this {country}")

capital_chain = LLMChain(llm=llm,prompt=capital_template)

famous_template = PromptTemplate(input_variables=['capital'],
    template = "Suggest me some amazing places to visit in {capital}")

famous_chain = LLMChain(llm=llm,prompt=famous_template)

In [30]:
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains=[capital_chain,famous_chain])
chain.run("India")

" It is a vibrant and diverse city with a rich history and culture. Here are some amazing places to visit in New Delhi:\n\n1. Red Fort - This iconic monument is a UNESCO World Heritage Site and was the residence of Mughal emperors for nearly 200 years. It is a stunning example of Mughal architecture and is a must-visit for history buffs.\n\n2. Qutub Minar - Another UNESCO World Heritage Site, this 73-meter tall minaret is the tallest brick minaret in the world. It is surrounded by other historical sites and is a popular spot for tourists.\n\n3. India Gate - This war memorial is a symbol of India's pride and is a popular spot for picnics and evening strolls. It is also surrounded by beautiful gardens and fountains.\n\n4. Lotus Temple - This Bahá'í House of Worship is a stunning architectural marvel and is known for its lotus-shaped design. It is a peaceful place for meditation and reflection.\n\n5. Humayun's Tomb - This 16th-century mausoleum is the final resting place of Mughal Emperor

## Sequential Chain

In [31]:
capital_template = PromptTemplate(input_variables=['country'],
    template = "Tell me the capital of this {country}")

capital_chain = LLMChain(llm=llm,prompt=capital_template,output_key="capital")

famous_template = PromptTemplate(input_variables=['capital'],
    template = "Suggest me some amazing places to visit in {capital}")

famous_chain = LLMChain(llm=llm,prompt=famous_template,output_key="places")


In [37]:
from langchain.chains import SequentialChain
chain = SequentialChain(chains=[capital_chain,famous_chain],
    input_variables=['country'],output_variables=['capital','places'])


In [38]:
chain({'country':'India'})

{'country': 'India',
 'capital': '\n\nThe capital of India is New Delhi.',
 'places': " Some amazing places to visit in New Delhi are:\n\n1. Red Fort - A UNESCO World Heritage Site and one of the most iconic monuments of Delhi, the Red Fort is a must-visit for its stunning architecture and rich history.\n\n2. Qutub Minar - Another UNESCO World Heritage Site, the Qutub Minar is the tallest brick minaret in the world and a prime example of Indo-Islamic architecture.\n\n3. Humayun's Tomb - A beautiful mausoleum built in the 16th century for the Mughal emperor Humayun, this monument is a blend of Persian and Indian architectural styles.\n\n4. India Gate - A war memorial and a popular spot for picnics and evening strolls, India Gate is a must-visit for its grandeur and historical significance.\n\n5. Lotus Temple - This magnificent temple, shaped like a lotus flower, is a symbol of peace and unity and is open to people of all faiths.\n\n6. Akshardham Temple - A modern marvel of architecture,

## Chatmodels with ChatOpenAI

In [39]:
from langchain.chat_models import ChatOpenAI

In [40]:
from langchain.schema import HumanMessage, SystemMessage, AIMessage

In [44]:
chatllm = ChatOpenAI(temperature=0.6,model='gpt-3.5-turbo')

In [45]:
chatllm([
    SystemMessage(content="Your are a comedian AI assitatnt"),
    HumanMessage(content="Please provide some comedy punchlines")
])

AIMessage(content='1. "I told my wife she should embrace her mistakes. So now she\'s wearing a T-shirt that says \'I\'m with Stupid\'."\n\n2. "I\'m not saying my house is messy, but I just found my car keys in the fridge. Apparently, my snacks are on the go now."\n\n3. "I tried to join a gym, but they said I needed a doctor\'s note. I guess they thought my lack of exercise was a medical condition called \'couch potato-itis\'."\n\n4. "I went to a restaurant that claimed to serve \'fast food\'. Well, their definition of fast must be different from mine because I waited so long that my hunger actually evolved into patience."\n\n5. "I recently started a new diet where I only eat things that are green. Turns out, green M&Ms don\'t count as a vegetable. Who knew?"\n\n6. "My friend asked me if I\'m good at math. I said, \'Well, I can count the number of times I\'ve failed math class on one hand.\'"\n\n7. "I went to a job interview and they asked me where I see myself in five years. I said, \'

### Prompt Template + LLM + Output Parsers

In [47]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

In [55]:
class Commaseparatedoutput(BaseOutputParser):
    def parse(self,text:str):
        return text.strip().split(',')

In [56]:
template = "You are a helpful assistant. When the user gives any input, you should generate 5 words in a comma separated list"
human_template = "{text}"
chatprompt = ChatPromptTemplate.from_messages([
    ("system",template),
    ("human",human_template)
])

In [58]:
chain = chatprompt|chatllm|Commaseparatedoutput()

In [59]:
chain.invoke({"text":"Intelligent"})

['Intelligent', ' Smart', ' Clever', ' Brilliant', ' Wise']